In [9]:
# from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
from selenium.webdriver import Firefox
import pandas as pd
import re
from datetime import datetime
# from bs4 import BeautifulSoup as bs
# import tqdm

In [13]:
filename = 'amazon_urls.txt'
file = open('amazon_urls.txt')

In [14]:
LIST=re.split("\r\n|\n",file.read())
file.close()

In [15]:
x = []
for i in LIST:
    print(i.split('~')[-1])
    x.append(i.split('~')[-1])
print(x)

https://www.amazon.com/Childrens-Place-Toddler-Cartwheel-Shorts/dp/B08Y1CDJJ2/
['https://www.amazon.com/Childrens-Place-Toddler-Cartwheel-Shorts/dp/B08Y1CDJJ2/']


In [16]:
filename.split(".")

['amazon_urls', 'txt']

In [17]:
len(x)


1

In [18]:
product_description, product_url, reviews, reviewers, reviewers_profile, isverified,reviewtitles,reviewedon,reviewerplace,reviewratings, prodname, dfs = [], [], [], [] ,[], [],[],[],[],[],[],[]
driver = Firefox()
df = pd.DataFrame()
driver.maximize_window()
i = 0
urlCounter = 0
while urlCounter != len(x):
    url = x[urlCounter]
    try:
        driver.get(url)
        product_name = url.split("/")[3].replace("-"," ")
        print(product_name, end='\n')
        prodname.append(product_name)
    except Exception as ex:
        print(ex)
    try:
        all_reviews = driver.find_element(By.CSS_SELECTOR,'#reviews-medley-footer .a-text-bold')
        product_details = driver.find_element(By.XPATH, '//div[@id="productDescription"]//span').text
        print(product_details)
        all_reviews.click()
    except Exception as ex:
        print(ex)
    while True:
        ratings = driver.find_elements(By.XPATH, '//i[@data-hook="review-star-rating"]/span')
        lst_of_ratings = driver.find_elements(By.CLASS_NAME, "review-rating")
        review_title = driver.find_elements(By.XPATH, "//a[@data-hook='review-title']/span")
        is_verified = driver.find_elements(By.XPATH, '//span[@data-hook="avp-badge"]')
        loc_date = driver.find_elements(By.XPATH, "//span[@data-hook='review-date']")
        review = driver.find_elements(By.XPATH, '//span[@data-hook="review-body"]')
        reviewer = driver.find_elements(By.CSS_SELECTOR, '#cm_cr-review_list .a-profile-name')
        reviewer_elements = driver.find_elements(By.CSS_SELECTOR, '#cm_cr-review_list .a-profile')
        try:
            next_btn = driver.find_element(By.CSS_SELECTOR, "#cm_cr-pagination_bar a")
        except:
            pass
        for i in range(len(review_title)):
            try:
                other, date = loc_date[i].text.split('on')
            except:
                pass
            place = other.split(' ')[3] +" "+ other.split(" ")[4]
            reviewedon.append(date)
            reviewerplace.append(place)

            for value in lst_of_ratings:
                try:
                    rating = value.get_attribute('textContent')
                except:
                    rating='Not Available!'
            reviewratings.append(rating)
            try:
                if len(is_verified) > 0:
                    isverified.append(is_verified[i].text)
                else:
                    isverified.append("Not Verified Purchase")
                reviewers_profile.append(reviewer_elements[i].get_attribute('href'))
                reviews.append(review[i].text)
                reviewers.append(reviewer[i].text)
                reviewtitles.append(review_title[i].text)
                product_description.append(product_details)
                product_url.append(url)
            except:
                pass
        try:
            next_btn = driver.find_element(By.XPATH, "//ul[@class='a-pagination']/li[2]/a")
            next_btn.click()
            time.sleep(8)
        except Exception as ex:
            break
    
    data = {'Reviewer':reviewers, 'Reviewer Profile Link':reviewers_profile, 'Review Title':reviewtitles, 'Is Verified':isverified,'Review Detail':reviews,'Date':reviewedon,'Place':reviewerplace,
       'Rating':reviewratings, 'Product Description':product_description, 'Product Link':product_url }
    min_length = len(reviewtitles)
    df = pd.DataFrame({k:pd.Series(v[:min_length]) for k,v in data.items()})
    df['Product Name'] = product_name
    dfs.append(df)
    del df
    i+=1
    urlCounter += 1
    
finaldata = pd.concat(dfs,axis=0)
current_time = datetime.now().strftime("%Y-%m-%d %H-%M-%S")
finaldata.to_csv('Amazon_data_'+current_time+'_.csv',index=False)
driver.close()
print("Finished")

Childrens Place Toddler Cartwheel Shorts
The Children’s Place offers a huge selection of kid's clothing! Shop us for jeans, shorts, leggings, chinos, polo shirts, dresses, uniforms, pajamas, and accessories. Made with premium quality materials at a great value. Kids clothing made simple.
Finished
